In [0]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

# To Do:
1. Separate the last 28 days.
    * We run everything as normal except:
      * During the training data set selection, we ignore the last 28 days
      * after the predictions are added(before converting to the F_ columns) we just extract the sales data for the last 28 days of our prediction and compare to the already made 28 days

2. Produce predictions without the last 28 days.
3. Compare predictions to the last 28 days.
4. Get a baseline for how well the default and magin number versions run (separate computers)
5. Remove random validation step.
6. Add 364 day lag and 14 day window
7. Convert all predictions < 0.01 to 0
8. Replace rolling mean with weighted mean


In [0]:
DIRECTORY = "/content/drive/My Drive/Colab Notebooks/lsda/HW1/"
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

# Changes (AUTHOR)
* v5 : try to optimise the LGBM params (go below in lgbm params section to see changes)
* v4 : add df, X_train deletion before training step --> increasing train sample without memeroy issues

# Changes (us)
1. Default no magic number (version 7) -> 0.55648

In [0]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [0]:
pd.options.display.max_columns = 50

In [0]:
h = 28 
max_lags = 57
tr_last = 1913-28 #CHANGE
fday = datetime(2016,4, 25) - timedelta(days=28) #CHANGE
fday

datetime.datetime(2016, 3, 28, 0, 0)

In [0]:
def create_dt(is_train = True, nrows = None, first_day = 1200, tr_last=tr_last):
    prices = pd.read_csv(DIRECTORY + "data/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(DIRECTORY + "data/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv(DIRECTORY + "data/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [0]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [0]:
FIRST_DAY = 350 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [0]:
df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

(11713404, 22)

In [0]:
df

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1500,0.0,2015-03-08,11506,3,2,3,2015,0,0,0,0,1.0,0.0,1.0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1501,0.0,2015-03-09,11506,1,3,3,2015,0,0,0,0,1.0,1.0,1.0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1502,0.0,2015-03-10,11506,5,4,3,2015,0,0,0,0,1.0,0.0,0.0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1503,1.0,2015-03-11,11506,6,5,3,2015,0,0,0,0,0.0,1.0,1.0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1504,1.0,2015-03-12,11506,4,6,3,2015,0,0,0,0,0.0,1.0,1.0,8.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11713399,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1885,1.0,2016-03-27,11609,3,2,3,2016,5,1,0,0,0.0,0.0,0.0,3.98
11713400,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1884,2.0,2016-03-26,11609,2,1,3,2016,0,0,0,0,0.0,0.0,0.0,1.28
11713401,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1885,4.0,2016-03-27,11609,3,2,3,2016,5,1,0,0,0.0,0.0,0.0,1.28
11713402,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1884,0.0,2016-03-26,11609,2,1,3,2016,0,0,0,0,0.0,0.0,0.0,1.00


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11713404 entries, 0 to 11713403
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory us

In [0]:
create_fea(df)
df

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1500,0.0,2015-03-08,11506,3,2,3,2015,0,0,0,0,1.0,0.0,1.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,10,1,8
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1501,0.0,2015-03-09,11506,1,3,3,2015,0,0,0,0,1.0,1.0,1.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,9
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1502,0.0,2015-03-10,11506,5,4,3,2015,0,0,0,0,1.0,0.0,0.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,10
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1503,1.0,2015-03-11,11506,6,5,3,2015,0,0,0,0,0.0,1.0,1.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,11
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1504,1.0,2015-03-12,11506,4,6,3,2015,0,0,0,0,0.0,1.0,1.0,8.26,NaN,NaN,NaN,NaN,NaN,NaN,11,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11713399,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1885,1.0,2016-03-27,11609,3,2,3,2016,5,1,0,0,0.0,0.0,0.0,3.98,0.0,0.0,1.000000,0.428571,1.178571,0.642857,12,1,27
11713400,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1884,2.0,2016-03-26,11609,2,1,3,2016,0,0,0,0,0.0,0.0,0.0,1.28,1.0,0.0,1.428571,0.000000,0.821429,1.000000,12,1,26
11713401,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1885,4.0,2016-03-27,11609,3,2,3,2016,5,1,0,0,0.0,0.0,0.0,1.28,1.0,0.0,1.142857,0.000000,0.857143,0.892857,12,1,27
11713402,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1884,0.0,2016-03-26,11609,2,1,3,2016,0,0,0,0,0.0,0.0,0.0,1.00,1.0,4.0,2.142857,1.571429,1.607143,1.607143,12,1,26


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11713404 entries, 0 to 11713403
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
 22  lag_7         float32       
 23  lag_28        float32       
 

In [0]:
df.dropna(inplace = True)
df.shape

(10036542, 31)

In [0]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [0]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [0]:
np.random.seed(42)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

In [0]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

43

In [0]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}


# params = {
#                     'boosting_type': 'gbdt',
#                     'objective': 'tweedie',
#                     'tweedie_variance_power': 1.1,
#                     'metric': 'rmse',
#                     'subsample': 0.5,
#                     'subsample_freq': 1,
#                     'learning_rate': 0.03,
#                     'num_leaves': 2**11-1,
#                     'min_data_in_leaf': 2**12-1,
#                     'feature_fraction': 0.5,
#                     'max_bin': 100,
#                     'n_estimators': 2,
#                     'boost_from_average': False,
#                     'verbose': -1,
#                 } 
VERSION = "350_days_MAGIC_NUMBER" # remove last 28 days

In [0]:
m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[20]	valid_0's rmse: 2.49528
[40]	valid_0's rmse: 2.21562
[60]	valid_0's rmse: 2.14482
[80]	valid_0's rmse: 2.12318
[100]	valid_0's rmse: 2.1137
[120]	valid_0's rmse: 2.10598
[140]	valid_0's rmse: 2.10052
[160]	valid_0's rmse: 2.09368
[180]	valid_0's rmse: 2.08861
[200]	valid_0's rmse: 2.08431
[220]	valid_0's rmse: 2.08149
[240]	valid_0's rmse: 2.07774
[260]	valid_0's rmse: 2.07497
[280]	valid_0's rmse: 2.07317
[300]	valid_0's rmse: 2.07121
[320]	valid_0's rmse: 2.0693
[340]	valid_0's rmse: 2.06734
[360]	valid_0's rmse: 2.06487
[380]	valid_0's rmse: 2.06226
[400]	valid_0's rmse: 2.0611
[420]	valid_0's rmse: 2.0594
[440]	valid_0's rmse: 2.05747
[460]	valid_0's rmse: 2.05587
[480]	valid_0's rmse: 2.05408
[500]	valid_0's rmse: 2.05206
[520]	valid_0's rmse: 2.0508
[540]	valid_0's rmse: 2.04961
[560]	valid_0's rmse: 2.04869
[580]	valid_0's rmse: 2.04741
[600]	valid_0's rmse: 2.04613
[620]	valid_0's rmse: 2.04472
[640]	valid_0's rmse: 2.04281
[660]	valid_0's rmse: 2.04162
[680]	valid_0's rms

In [0]:
#2.30653 DEFAULT NO MAGIC NUMBER

In [0]:
m_lgb.save_model(DIRECTORY + "models/modelv_" + str(VERSION) + ".lgb")

In [0]:
# #load model
# m_lgb = lgb.Booster(model_file=DIRECTORY + "models/model.lgb")
# VERSION = "BOOSTER_model"

In [0]:
alphas = [1.028, 1.023, 1.018] #original
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    #te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv(DIRECTORY + "submissions/submission_v" + str(VERSION) + ".csv",index=False)

0 2016-03-28 00:00:00
1 2016-03-29 00:00:00
2 2016-03-30 00:00:00
3 2016-03-31 00:00:00
4 2016-04-01 00:00:00
5 2016-04-02 00:00:00
6 2016-04-03 00:00:00
7 2016-04-04 00:00:00
8 2016-04-05 00:00:00
9 2016-04-06 00:00:00
10 2016-04-07 00:00:00
11 2016-04-08 00:00:00
12 2016-04-09 00:00:00
13 2016-04-10 00:00:00
14 2016-04-11 00:00:00
15 2016-04-12 00:00:00
16 2016-04-13 00:00:00
17 2016-04-14 00:00:00
18 2016-04-15 00:00:00
19 2016-04-16 00:00:00
20 2016-04-17 00:00:00
21 2016-04-18 00:00:00
22 2016-04-19 00:00:00
23 2016-04-20 00:00:00
24 2016-04-21 00:00:00
25 2016-04-22 00:00:00
26 2016-04-23 00:00:00
27 2016-04-24 00:00:00
0 1 0.3333333333333333
0 2016-03-28 00:00:00
1 2016-03-29 00:00:00
2 2016-03-30 00:00:00
3 2016-03-31 00:00:00
4 2016-04-01 00:00:00
5 2016-04-02 00:00:00
6 2016-04-03 00:00:00
7 2016-04-04 00:00:00
8 2016-04-05 00:00:00
9 2016-04-06 00:00:00
10 2016-04-07 00:00:00
11 2016-04-08 00:00:00
12 2016-04-09 00:00:00
13 2016-04-10 00:00:00
14 2016-04-11 00:00:00
15 2016-

In [0]:
sub

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.577977,0.558246,0.616576,0.768015,0.922768,1.128305,0.870042,0.693204,0.715720,0.691339,0.700831,0.901822,1.141889,0.956231,0.837839,0.786378,0.770848,0.780016,0.872034,1.178456,0.997718,0.687093,0.675743,0.600196,0.666420,0.878599,1.115616,1.016128
1,FOODS_1_001_CA_2_validation,0.859451,0.821642,0.962451,1.197356,1.414802,1.769074,1.438902,0.919244,0.941878,1.075573,1.067401,1.334879,2.114438,1.589644,1.149552,1.073136,0.998851,1.042058,1.274919,1.554381,1.441401,0.819714,0.728956,0.728984,0.753017,1.104950,1.518628,1.280096
2,FOODS_1_001_CA_3_validation,0.847483,0.717845,0.826661,1.024479,1.190517,1.365071,1.546702,0.968570,0.988431,0.936814,0.961627,1.343741,2.311453,1.595986,1.468445,1.266447,1.130522,1.090219,1.631627,2.130859,1.674649,0.984308,0.868333,0.852552,0.814994,1.270521,1.419071,1.279547
3,FOODS_1_001_CA_4_validation,0.381060,0.377130,0.473346,0.538226,0.474603,0.542036,0.448550,0.402831,0.448874,0.430791,0.463127,0.497878,0.610491,0.475027,0.410071,0.401943,0.399151,0.418924,0.435698,0.462929,0.406277,0.346696,0.341595,0.365931,0.359224,0.404610,0.439178,0.392758
4,FOODS_1_001_TX_1_validation,0.328469,0.355217,0.430532,0.524039,0.748909,0.718163,0.621199,0.528297,0.527771,0.532966,0.515909,0.598135,0.865461,0.605329,0.513384,0.494585,0.449989,0.482290,0.554246,0.694687,0.553695,0.428724,0.406485,0.404116,0.377102,0.488756,0.582447,0.495532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0.275639,0.239537,0.284751,0.334518,0.359787,0.431346,0.329945,0.231905,0.223135,0.223181,0.261429,0.326059,0.401336,0.412289,0.282616,0.269956,0.265960,0.274059,0.348907,0.469560,0.422916,0.313824,0.292400,0.277760,0.316517,0.355072,0.463698,0.452943
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.158007,0.146247,0.159766,0.177198,0.193177,0.272969,0.236860,0.184148,0.155563,0.179488,0.184997,0.197916,0.254967,0.200199,0.159534,0.148982,0.144964,0.150624,0.171010,0.233574,0.196764,0.145159,0.136029,0.138740,0.153473,0.155293,0.225372,0.206751
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.076718,0.071721,0.081047,0.092660,0.118259,0.125405,0.102423,0.070289,0.064148,0.065094,0.073254,0.099207,0.121409,0.112846,0.066867,0.066214,0.068150,0.077022,0.101864,0.125021,0.115109,0.068932,0.065293,0.066339,0.079107,0.107430,0.124342,0.120962
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.059827,0.059236,0.067078,0.077784,0.089286,0.094924,0.078442,0.063184,0.064529,0.064793,0.068404,0.088333,0.091125,0.085547,0.064511,0.065256,0.063578,0.068990,0.085965,0.089047,0.081350,0.059076,0.058625,0.058205,0.065838,0.088055,0.087614,0.088430


In [0]:
from sklearn.metrics import mean_squared_error
#last_28 = pd.read_csv(DIRECTORY + "data/sales_last28.csv") #BACKUP TRUE DATASET

def create_test():
  """
  Create test set starting from last day + 1 used for
  trainining. We split our trining data into test and train,
  we use the last 28 days as test set, and all the previous
  as train set. Last day of train data is 1913 - 28,
  first day of test is 1913 - 28 + 1 )
  """
  df_test2 = create_dt(is_train=True, first_day= 1913-27, tr_last=1913)
 
  cols = [f"F{i}" for i in range(1,29)]
 
  df_test2["F"] = [f"F{rank}" for rank in df_test2.groupby("id")["id"].cumcount()+1]
  df_test2 = df_test2.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
  df_test2.fillna(0., inplace = True)
  df_test2.sort_values("id", inplace = True)
  df_test2.reset_index(drop=True, inplace = True)
  return df_test2

def is_testset_correct(test_set):
  """
  Run only if check is needed, memory consumption.
  Given a test_set (in submission format) we created,
  we check if it actually match the last 28 days of
  the true sales train validation data.
  Return 0 if perfect match.
  """
  # Load dataset (takes memory)
  true_sales = pd.read_csv(DIRECTORY + "data/sales_train_validation.csv")
  dcols = [f"d_{i}" for i in range(1913-27, 1914)]
  fcols = [f"F{i}" for i in range(1, 29)]
  true_sales = true_sales[["id"] + dcols]
  true_sales.sort_values("id", inplace = True)
 
  # Check if is correct
  test_set = test_set[["id"] + fcols]
  value = np.sum(test_set[fcols].values - true_sales[dcols].values)
  return value
 
#print(is_testset_correct(create_test()))


def get_rmse(predictions):
  """
  Given a prediction (submission format),
  return RMSE using the create_test function
  (last 28 days of the sales_train_validation.csv
  in submission format)
  """
  # Take all columns beside ID for both test and pred data and extract values
  #test_data = create_test().iloc[:,1:]
  test_data = last_28.iloc[:,1:] 
  print(test_data.shape)
  predictions = predictions.iloc[:,1:] + 1.5
  print(predictions.shape)
  error = mean_squared_error(predictions, test_data, squared=False)
  #error = test_data.values - predictions.values
  return error
 
get_rmse(sub.iloc[:30490])


(30490, 28)
(30490, 28)


2.629865176040201

In [0]:
# current_submission = pd.read_csv(DIRECTORY + "submissions/submission_v1500days_split.csv")
# current_submission = current_submission[:30490]
# get_rmse(current_submission)

(30490, 28)
(30490, 28)


2.115990727981268

In [0]:
# kaggle_submission = pd.read_csv(DIRECTORY + "submissions/submissionv7.csv")
# kaggle_submission = kaggle_submission.iloc[:30490]
# get_rmse(kaggle_submission)

(30490, 28)
(30490, 28)


1.9046647031682857

In [0]:
# x = sub.iloc[:30490]
# x.loc[x.id == "FOODS_3_827_WI_3_validation"]

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
14369,FOODS_3_827_WI_3_validation,1.312158,1.22073,1.35381,1.688949,2.519725,2.215919,2.834552,1.824757,1.883377,1.663044,1.513115,2.069368,2.109087,1.921346,1.885157,1.748202,1.71515,1.861155,2.690923,2.089474,2.06552,1.65756,1.43583,1.496082,1.530962,2.041069,1.818891,2.084085


In [0]:
#FIRST MODEL (TOTAL AVERAGE)
################################
sales_train = pd.read_csv(DIRECTORY + "data/sales_train_validation.csv")
sample_output = pd.read_csv(DIRECTORY + "data/sample_submission.csv")

d_cols = [col for col in sales_train.columns if 'd_' in col]
f_cols = sample_output.columns[1:]
#get means
means = sales_train[d_cols[-30:]].mean(axis = 1)
means = sales_train[d_cols].mean(axis = 1)

for f in f_cols:
    sample_output[f] = means
sample_output = sample_output.fillna(0)
get_rmse(sample_output[:30490])
#sample_output.to_csv("submission.csv", index = False)

(30490, 28)
(30490, 28)


4.502574763513422

In [0]:
true_values = create_test()


(30490, 28)
(30490, 28)


0.0

In [0]:
get_rmse(true_values)

(30490, 28)
(30490, 28)


1.5

In [0]:
#true_values = create_test()
#true_values#.loc[true_values.id == "FOODS_3_827_WI_3_validation"]

In [0]:
#te#.loc[te.id == "HOBBIES_1_001_CA_1_validation"].sales
tst

,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
2561049,0,0,0,0,0,2,4,2016,0,0,0,0,0.0,0.0,0.0,8.38,1.254725,1.0,1.050753,1.142857,1.039924,1.000000,16,2,24
2561051,1,0,0,0,0,2,4,2016,0,0,0,0,0.0,0.0,0.0,3.97,0.438366,1.0,0.323716,0.571429,0.370842,0.321429,16,2,24
2561053,2,0,0,0,0,2,4,2016,0,0,0,0,0.0,0.0,0.0,2.97,0.750225,0.0,0.523549,0.285714,0.414046,0.535714,16,2,24
2561055,3,0,0,0,0,2,4,2016,0,0,0,0,0.0,0.0,0.0,4.64,3.109369,6.0,2.211552,3.571429,2.504565,1.892857,16,2,24
2561057,4,0,0,0,0,2,4,2016,0,0,0,0,0.0,0.0,0.0,2.88,1.462526,0.0,1.155827,0.857143,1.086680,1.107143,16,2,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622019,3044,6,9,2,2,2,4,2016,0,0,0,0,0.0,0.0,0.0,2.98,0.347495,0.0,0.338687,0.000000,0.209446,0.250000,16,2,24
2622021,3045,6,9,2,2,2,4,2016,0,0,0,0,0.0,0.0,0.0,2.48,0.086318,0.0,0.105936,0.000000,0.056533,0.000000,16,2,24
2622023,3046,6,9,2,2,2,4,2016,0,0,0,0,0.0,0.0,0.0,3.98,1.011519,1.0,1.006360,0.714286,0.925104,1.250000,16,2,24
2622025,3047,6,9,2,2,2,4,2016,0,0,0,0,0.0,0.0,0.0,1.28,1.223270,4.0,1.230905,1.571429,1.286602,1.250000,16,2,24


In [0]:

# te = create_dt(False)
# cols = [f"F{i}" for i in range(1,29)]

# for tdelta in range(0, 28):
#     day = fday + timedelta(days=tdelta)
#     print(tdelta, day)
#     tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
#     create_fea(tst)
#     tst = tst.loc[tst.date == day , train_cols]
#     te.loc[te.date == day, "sales"] = m_lgb.predict(tst)



# te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
# te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
# te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
# te_sub.fillna(0., inplace = True)
# te_sub.sort_values("id", inplace = True)
# te_sub.reset_index(drop=True, inplace = True)



#te_sub.to_csv("submission_v" + str(VERSION) + ".csv",index=False)

In [0]:
# te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
# te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
# te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
# te_sub.fillna(0., inplace = True)
# te_sub.sort_values("id", inplace = True)
# te_sub.reset_index(drop=True, inplace = True)


In [0]:
# VERSION = 7
# sub = te_sub
# sub2 = sub.copy()
# sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
# sub = pd.concat([sub, sub2], axis=0, sort=False)
# sub.to_csv(DIRECTORY + "submissions/submission_v" + str(VERSION) + ".csv",index=False)

In [0]:

#te_sub.to_csv(DIRECTORY + "submissions/submission_v" + str(VERSION) + ".csv",index=False)

In [0]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

In [0]:
sub.shape

In [0]:
# !wc -l submission_0.csv
# !head submission_0.csv

In [0]:
# !wc -l /content/drive/My\ Drive/Colab\ Notebooks/lsda/HW1/submissions/submission_v3.csv
# !head /content/drive/My\ Drive/Colab\ Notebooks/lsda/HW1/submissions/submission_v3.csv
!ls /content/drive/My\ Drive/Colab\ Notebooks/lsda/HW1/submissions/

In [0]:
# x = pd.read_csv("/content/drive/My Drive/Colab Notebooks/lsda/HW1/submissions/submission_v6.csv")
# y = pd.read_csv("/content/drive/My Drive/Colab Notebooks/lsda/HW1/submissions/submissionv7.csv")
# x.iloc[:,1:]-y.iloc[:,1:]